### Preparation of Training and Validation Sets

In [1]:
import os

In [2]:
lesson_dir = os.getcwd()
data_dir = os.path.join(lesson_dir, 'data')
if not os.path.isdir(data_dir): os.mkdir(data_dir)

In [12]:
train_dir = os.path.join(data_dir, 'train')
categories = [dir for dir in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, dir))]

In [41]:
for directory in ['full', 'sample']:
    for subdirectory in ['train', 'validation']:
        for category in categories:
            full_dir = os.path.join(data_dir, directory, subdirectory, category)
            if not os.path.isdir(full_dir): os.makedirs(full_dir)

In [14]:
def copyfiles(list_file_paths, target_dir):
    for path in list_file_paths: shutil.copy(path, target_dir)

In [42]:
import shutil
import glob
from sklearn.cross_validation import train_test_split
sample_size = 10
for category in categories:
    img_files = glob.glob(os.path.join(train_dir, category, 'img*'))
    train_img, val_img = train_test_split(img_files, test_size=0.15)
    copyfiles(train_img, os.path.join(data_dir, 'full', 'train', category))
    copyfiles(val_img, os.path.join(data_dir, 'full', 'validation', category))
    copyfiles(train_img[:sample_size], os.path.join(data_dir, 'sample', 'train', category))
    copyfiles(val_img[:sample_size], os.path.join(data_dir, 'sample', 'validation', category))

### Load VGG16 Model and Weights

In [27]:
import sys
sys.path.insert(1, '../')
from vgg16 import Vgg16
from utils import *

Using Theano backend.


In [64]:
#model_data_dir = os.path.join(data_dir, 'sample')
model_data_dir = os.path.join(data_dir, 'full')

In [43]:
vgg16 = Vgg16()

/Users/kwanchongtan/anaconda/lib/python3.5/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [44]:
def pop_model_layers(model, nlayers):
    for i in range(nlayers):
        model.pop()

In [45]:
pop_model_layers(vgg16.model, 2)

### Get VGG16 Features

In [65]:
train_batches = get_batches(os.path.join(model_data_dir, 'train'), shuffle=False, batch_size=1, target_size=(224,224))
val_batches = get_batches(os.path.join(model_data_dir, 'validation'), shuffle=False, batch_size=1, target_size=(224,224))

Found 19056 images belonging to 10 classes.
Found 3368 images belonging to 10 classes.


In [66]:
train_labels = onehot(train_batches.classes)
val_labels = onehot(val_batches.classes)

In [67]:
train_features = vgg16.model.predict_generator(train_batches, train_batches.N)

In [68]:
val_features = vgg16.model.predict_generator(val_batches, val_batches.N)

In [69]:
val_features.shape

(3368, 4096)

In [70]:
val_labels.shape

(3368, 10)

In [71]:
from keras.models import Sequential
lm = Sequential()
lm.add(Dense(10, activation="softmax", input_shape=(4096,)))
lm.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [72]:
lm.fit(train_features, train_labels, batch_size=len(train_features), nb_epoch=1, validation_data=(val_features, val_labels));

Train on 19056 samples, validate on 3368 samples
Epoch 1/1
19056/19056 [==============================] - 0s - loss: 3.2545 - acc: 0.1169 - val_loss: 3.8740 - val_acc: 0.1217
